# Pre processing

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?code_challenge=KxYCV1gmQpTcLiDIcN0lrDP__R9f9KsvS7yp8lfnkkk&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


Enter verification code: 4/uwHg5xwUuukbTW1yDSLeC8oYq7b4xkhEMa9WWTKJb4WGEzKNerp4N-8
If you need to use ADC, see:
  gcloud auth application-default --help

You are now logged in as [galli.giuly@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [0]:
%env GCLOUD_PROJECT=reddit-master

env: GCLOUD_PROJECT=reddit-master


## Standardization 

In [0]:
import pandas as pd
import pickle 

from keras.preprocessing.text import Tokenizer
from sklearn import preprocessing
from nltk import word_tokenize


Using TensorFlow backend.


In [0]:
# importing my final ds

!gsutil cp gs://reddit_final_results/reddit_clear_df.pkl .

Copying gs://reddit_final_results/reddit_clear_df.pkl...
- [1 files][253.2 MiB/253.2 MiB]                                                
Operation completed over 1 objects/253.2 MiB.                                    


In [0]:
reddit_clear_df = pd.read_pickle('reddit_clear_df.pkl')


### Encoding the label

In case I would need a encoded label to train my final model, I prepare the data set also with `subreddit_id` and I save it as `reddit_clear_encoded_df`

In [0]:
# Creting the new df with the encoded column 

reddit_clear_encoded_df = reddit_clear_df

In [0]:
reddit_clear_encoded_df.head()

,subreddit,body
0,gaming,honestly share height rest shorties
1,gaming,im sure whats funnier way interpreted said act...
2,gaming,wait quelaags furysword scaled soft humanity t...
3,gaming,program use make display maps something simila...
4,gaming,ccleaner turned shit norton makes tool uninsta...


In [0]:
# Encode of the label as subreddit_it

reddit_clear_encoded_df['subreddit_id'] = reddit_clear_encoded_df['subreddit']

In [0]:
reddit_clear_encoded_df.head(5)

,subreddit,body,subreddit_id
0,gaming,honestly share height rest shorties,gaming
1,gaming,im sure whats funnier way interpreted said act...,gaming
2,gaming,wait quelaags furysword scaled soft humanity t...,gaming
3,gaming,program use make display maps something simila...,gaming
4,gaming,ccleaner turned shit norton makes tool uninsta...,gaming


In [0]:
le = preprocessing.LabelEncoder()

In [0]:
le.fit(reddit_clear_encoded_df['subreddit'])

LabelEncoder()

In [0]:
list(le.classes_)

['Fitness',
 'atheism',
 'aww',
 'europe',
 'gaming',
 'movies',
 'nba',
 'politics',
 'science',
 'technology']

In [0]:
reddit_clear_encoded_df['subreddit_id'] = le.transform(reddit_clear_encoded_df['subreddit']) 

In [0]:
reddit_clear_encoded_df.head()

,subreddit,body,subreddit_id
0,gaming,honestly share height rest shorties,4
1,gaming,im sure whats funnier way interpreted said act...,4
2,gaming,wait quelaags furysword scaled soft humanity t...,4
3,gaming,program use make display maps something simila...,4
4,gaming,ccleaner turned shit norton makes tool uninsta...,4


In [0]:
reddit_clear_encoded_df.to_pickle('reddit_clear_encoded_df.pkl')

In [0]:
!gsutil cp reddit_clear_encoded_df.pkl gs://reddit_final_results

Copying file://reddit_clear_encoded_df.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\
Operation completed over 1 objects/265.0 MiB.                                    


### Tokenization

In [0]:
MAX_NB_WORDS = 75000
MAX_SEQUENCE_LENGTH = 450
EMBEDDING_DIM = 100

In [0]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,)
tokenizer.fit_on_texts(reddit_clear_df['body'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 864049 unique tokens.


In [0]:
with open('reddit_tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)
    
!gsutil cp reddit_tokenizer.pkl gs://reddit_models/

Copying file://reddit_tokenizer.pkl [Content-Type=application/octet-stream]...
\
Operation completed over 1 objects/47.0 MiB.                                     
